In [1]:
import torch
from torch.nn import functional as F
from torch import nn
from pytorch_lightning.core.lightning import LightningModule

from ilan_src.models import *

In [ ]:
class WGANGP(LightningModule):
    def __init__(self,): # fill in
        super().__init()
        self.lr, self.b1, self.b2 = 
        self.disc_freq, self.gen_freq = 5, 1
        self.noise_shape = 
        self.lambda_gp = 
        self.gen = Generator(...)
        self.disc = Discriminator(...)
        
    def forward(self, condition, noise):
        return self.gen(condition, noise)
    
    def gradient_pentalty(self, condition, real, fake):
        BATCH_SIZE, C, H, W = real.shape
        epsilon = torch.rand((BATCH_SIZE, 1, 1, 1)).repeat(1,C,H,W).to(device)
        interpolated_images = real*epsilon + fake*(1-epsilon)

        mixed_scores = self.disc(condition, interpolated_images)
        gradient = torch.autograd.grad(
                    inputs=interpolated_images,
                    outputs=mixed_scores, 
                    grad_outputs = torch.ones_like(mixed_scores), 
                    create_graph=True, 
                    retain_graph = True)[0]

        gradient = gradient.view(gradient.shape[0], -1)
        gradient_norm = gradient.norm(2, dim=1)
        gradient_penalty = torch.mean((gradient_norm - 1)**2)
        return gradient_penalty
    
    def training_step(self, batch, batch_idx, optimizer_idx):
        real, condition = batch # if label is condition.
        
        # train discriminator
        if optimizer_idx == 0:
            noise = torch.randn(real.shape[0], *self.noise_shape)
            fake = self.gen(condition, noise)
            disc_real = self.disc(condition, real).reshape(-1)
            disc_fake = self.disc(condition, fake).reshape(-1)
            gp = self.gradient_penalty(condition, real, fake)
            loss_disc = -(torch.mean(disc_real) - torch.mean(disc_fake)) + self.lambda_gp*gp
            return loss_disc
        
        #train generator
        elif optimizer_idx ==1:
            noise = torch.randn(real.shape[0], *self.noise_shape)
            fake = self.gen(condition, noise)
            gen_fake = self.disc(condition, fake).reshape(-1)
            loss_gen = -torch.mean(gen_fake)
            return loss_gen        
        
    def configure_optimizers(self):
        gen_opt = optim.Adam(self.disc.parameters(), lr=self.lr, betas=(self.b1, self.b2))
        disc_opt = optim.Adam(self.disc.parameters(), lr=self.lr, betas=(self.b1, self.b2))
        return [{"optimizer": disc_opt, "frequency": self.disc_freq}, {"optimizer": gen_opt, "frequency": self.gen_freq}]
        